In [ ]:
def post_pruning_decision_tree_selector(df, y_label, n_bins = 5, cv = None, scoring = "accuracy"):
    X = df.drop(y_label,axis=1)
    y = df[y_label]
    y = bin_this(y, n_bins)
    
    clf = DecisionTreeClassifier(random_state=0)
    path = clf.cost_complexity_pruning_path(X, y)
    ccp_alphas, impurities = path.ccp_alphas, path.impurities
    
    parameters = {'ccp_alpha':ccp_alphas}

    gsCV = GridSearchCV(DecisionTreeClassifier(random_state=0),parameters, cv = cv, scoring=scoring, n_jobs=-1)
    gsCV.fit(X,y)
    clf = gsCV.best_estimator_
    print (clf.best_score_, clf.best_params_)
    return clf, clf.best_score_, clf.best_params_
    #clfs = []
    #scores = []
    #for ccp_alpha in ccp_alphas:
    #    # seriously not sure about this
    #    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    #    score = cross_val_score(estimator = clf, X = X, y = y, cv = cv, scoring=accuracy, n_jobs = -1)
    #    clf.fit(X,y)
    #    clfs.append(clf)
    #    scores.append(score)
    #return clfs, ccp_alphas, scores

def bin_this(y, n_bins = 5):
    # same sized bins
    if type(y[0]) == np.int_ or type(y[0]) == np.float_:
        y_range = y.max() - y.min()
        bin_names = []
        for n in range(n_bins):
            bin_names.append([y.min() + (n/n_bins)*y_range,y.min() + ((n+1)/n_bins)*y_range])
        y_new = np.array([])

        for ix, x in enumerate(y):
            for bin_name in bin_names:
                if(bin_name[0] <= x and x < bin_name[1]):
                    y_new = np.append(y_new, str(bin_name))
            if (x == y.max()):
                y_new = np.append(y_new, str(bin_names[n_bins - 1]))               
        return y_new
    else: # for strings and arrays # not tested yet
        return [str(x) for x in y]

In [ ]:
dataf = msoa_combined_year[ethnicities + ["energy_alcohol"]]
clf, score, alpha = post_pruning_decision_tree_selections(dataf, y_label = "energy_alcohol")

In [ ]:
text_representation = export_text(clf)
print(text_representation)

In [ ]:
fig = plt.figure(figsize=(15,10))
viz = plot_tree(clf)